In [2]:
import os
import pdb

import pandas as pd
from datasets import load_dataset
import cohere
from dotenv import load_dotenv



/home/sam/code/cohere-reasoning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Retrieve your API key from environment variables, and create a Cohere client
load_dotenv()
key = os.getenv("COHERE_API_KEY")
co = cohere.Client(key)

# Test the API
print(co.chat(message="Hello, world!"))

text='Hello! How can I help you today?' generation_id='fc6551f7-d9b5-4f3b-9a04-4a3b1dbbe42a' citations=None documents=None is_search_required=None search_queries=None search_results=None finish_reason='COMPLETE' tool_calls=None chat_history=[UserMessage(role='USER', message='Hello, world!', tool_calls=None), ChatbotMessage(role='CHATBOT', message='Hello! How can I help you today?', tool_calls=None)] prompt=None meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=4.0, output_tokens=9.0, search_units=None, classifications=None), tokens=ApiMetaTokens(input_tokens=210.0, output_tokens=9.0), warnings=None) response_id='2e88872b-5505-4b5d-a35e-eae3cad3b1c0'


In [4]:
# Download the 859594-record (859494 train, 100 test) NuminaMath-CoT Dataset from HF
# This dataset contains math problems with solutiosn formulated in a CoT fashion
# Sources range from Chinese high-school math to IMO problems
# The "cn_k12" Chinese high-school subset contains the 276591 records we'll use
dataset_name = "AI-MO/NuminaMath-CoT"
dataset = load_dataset(dataset_name)


In [5]:
# Combine the train/test splits, with a new column indicating original split
train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["test"])
train_df["set"] = "train"
test_df["set"] = "test"
df = pd.concat([train_df, test_df])


The Dataframe contains the following columns
- **source**: The source of the problem
- **problem**: The problem statement
- **solution**: The solution to the problem
- **messages**: problem/solution columns reformatted in a JSON chat-assitant format
- **set**: The original split of the problem (train or test)


In [5]:
# Filter to only the cn_k12 subset
df = df[df["source"] == "cn_k12"]
questions = df["problem"]

In [1]:
questions.iloc[0]

NameError: name 'questions' is not defined